# Процеси в Python

## Пакет multiprocessing

Пакет [multiprocessing](https://docs.python.org/3.8/library/multiprocessing.html) — це пакет для виконання коду в окремих процесах з інтерфейсом подібним до інтерфейсу пакета `threading`.

Основна причина появи `multiprocessing` — це `GIL (Global Interpreter Lock)` і той факт, що `threading API` не дозволяє розпаралелювати `CPU-bound `завдання. 
- Оскільки **в один момент часу завжди виконується код тільки в одному потоці**, навіть на багатоядерних сучасних процесорах, отримати приріст продуктивності для завдань, пов'язаних з інтенсивними обчисленнями, за допомогою `threading` не вийде.

Щоб виконувати обчислення дійсно паралельно там, де це дозволяє обладнання, в `Python` використовуються окремі процеси. Так, у кожному окремому процесі буде запущено свій інтерпретатор `Python` зі своїм `GIL`.

Для використання процесів необхідно імпортувати клас `Process` модуля `multiprocessing`. 

З ним можна працювати декількома способами:

- У процесі створення екземпляра класу `Process` іменованому аргументу `target` передати функцію, яка буде виконуватися в окремому процесі
- Реалізувати похідний клас від класу `Process` та перевизначити метод `run`

Розглянемо приклад:

У цьому прикладі ми створили п'ять процесів, у трьох з яких виконали функцію `example_work`, а у двох — це клас `MyProcess`, який наслідується від класу `Process`. 
- У процесів є код завершення роботи **(0 означає успішне завершення роботи у штатному режимі)**. 
- І після завершення роботи атрибут `exitcode` містить код завершення. В іншому `API multiprocessing` багато в чому повторює threading.

In [3]:
from multiprocessing import Process
import logging
from time import sleep

logger = logging.getLogger()
stream_handler = logging.StreamHandler()
logger.addHandler(stream_handler)
logger.setLevel(logging.DEBUG)


class MyProcess(Process):
    def __init__(self, group=None, target=None, name=None, args=(), kwargs=None, *, daemon=None):
        super().__init__(group=group, target=target, name=name, daemon=daemon)
        self.args = args

    def run(self) -> None:
        logger.debug(self.args)


def example_work(params):
    sleep(0.5)
    logger.debug(params)


if __name__ == '__main__':
    processes = []
    for i in range(3):
        pr = Process(target=example_work, args=(f"Count process function - {i}", ))
        pr.start()
        processes.append(pr)

    for i in range(2):
        pr = MyProcess(args=(f"Count process class - {i}",))
        pr.start()
        processes.append(pr)

    [el.join() for el in processes]
    [print(el.exitcode, end=' ') for el in processes]
    logger.debug('End program')



Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
Traceback (most recent call last):
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
  File "<string>", line 1, in <module>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    e

1 1 1 1 1 

[Загальні зауваження та поради](https://docs.python.org/3.8/library/multiprocessing.html#programming-guidelines) при роботі з процесами

## Contexts and start methods

Залежно від платформи `multiprocessing` підтримує 3 способи створення нового процесу:

- `spawn` — запускає новий процес `Python`, наслідуються лише ресурси, необхідні для запуску `run()`. Присутній в Unix і Windows. Є способом за замовчуванням для Windows і macOS.
- `fork` — дочірній процес, що є точною копією батьківського (включаючи всі потоки), доступний тільки на Unix. За замовчуванням використовується на Unix. Зробити безпечний `fork` досить проблематично і це може бути причиною неочевидних проблем.
- `forkserver` — створюється процес-фабрика (сервер для породження процесів за запитом). Наслідуються тільки необхідні ресурси, що використовуються `fork` для запуску нового процесу, але завдяки однопотоковій реалізації процесу-фабрики, це робиться безпечно. Доступний тільки на Unix платформах з підтримкою передачі файлових дескрипторів через pipes (що може суперечити безпековій політиці на багатьох системах).

Для вибору методу використовується `multiprocessing.set_start_method(method)

```python
import multiprocessing
...

if __name__ == '__main__':
    multiprocessing.set_start_method('forkserver')
    ...
```



## Interprocess communication

Для міжпроцесорної взаємодії виокремлюють наступні інструменти:
- файли;
- сокети;
- канали (всі POSIX ОС);
- роздільна пам'ять (всі POSIX ОС);
- семафори (всі POSIX ОС);
- сигнали або переривання (крім Windows);
- семафори (всі POSIX ОС);
- черга повідомлень;

- Найбільшу складність при роботі з процесами представляє **обмін даними між процесами**, оскільки у кожного процесу своя ізольована область пам'яті. 
- Механізми обміну даними залежать від ОС (Операційної Системи). 
     - Найуніверсальніший — `файли`. 
     - Але ви також можете скористатися мережевими інтерфейсами (`localhost`), 
     - примітивами на основі мережевих інтерфейсів (`pipe`) 
     - та `загальною пам'яттю`, де це можливо.

У будь-якому разі, крім загальної пам'яті, для обміну даними між процесами всі об'єкти серіалізуються та десеріалізуються. Цей додатковий крок створює навантаження на `CPU`.

**Найшвидшим та найекономнішим з погляду ресурсів способом обміну даними є спільна пам'ять.**

## Спільна пам'ять

Спільна пам'ять підтримується не всіма операційними системами та може бути заборонена політикою безпеки.

- Щоб створити область спільної пам'яті, **потрібно вказати ОС, скільки пам'яті потрібно виділити**. 
- Для обчислення обсягу пам'яті обов'язково потрібно **вказати тип даних**, який буде використовуватись, 
- та кількість елементів для складних типів. 
- Крім того, **механізми обмеження доступу до спільного ресурсу** також потрібно забезпечити самостійно, інакше дані можна зіпсувати при спробі одночасного доступу для зміни із різних процесів.

- У цьому прикладі ми скористалися **механізмом спільної пам'яті `Value`**. 
- Тип даних було обрано десятковий (`'d'`). Докладніше про доступні типи та їх назви можна дізнатися з документації.

Щоб створити спільну пам'ять для процесів, необхідно визначити тип даних, який буде знаходитися у виділеній області пам'яті. Це порушує звичний для Python підхід, коли не потрібно думати про те, який тип даних буде використовуватися і скільки місця він може займати.


```python
from multiprocessing import Process, Value, RLock, current_process
from time import sleep
import logging
import sys

logger = logging.getLogger()
stream_handler = logging.StreamHandler()
logger.addHandler(stream_handler)
logger.setLevel(logging.DEBUG)


def worker(val: Value):
    logger.debug(f'Started {current_process().name}')
    sleep(1)
    with val.get_lock():
        val.value += 1
    logger.debug(f'Done {current_process().name}')
    sys.exit(0)


if __name__ == '__main__':
    lock = RLock()
    value = Value('d', 0, lock=lock)
    pr1 = Process(target=worker, args=(value, ))
    pr1.start()
    pr2 = Process(target=worker, args=(value, ))
    pr2.start()

    pr1.join()
    pr2.join()

    print(value.value)  # 2.0
```

Виведення:

```
Started Process-1
Started Process-2
Done Process-1
Done Process-2
2.0
```

Давайте розберемо складніший приклад, з використанням структур у спільній пам'яті:

У цьому прикладі ми створили:

- структуру `Point`, яка описує координати точки на площині;
- дробове число `number`;
- рядкову змінну `string` (підтримуються тільки byte-рядки);
- масив `array`, який містить координати точок відповідно до структури `Point`.

Зверніть увагу, для опису полів структури їх потрібно помістити в список кортежів `_fields_`, `де кожен кортеж — це ім'я та тип поля`.

Масив `Array` поводиться багато в чому як список і дозволяє зберігати в ньому різнотипні дані, але його розмір статичний і додавати/видаляти елементи не можна. Так само як і змінювати тип існуючих.

Також у структурі даних ми передали механізм блокування через параметр `lock`. Як `Value`, так і `Array` забезпечують блокування ресурсу, до якого можна отримати доступ, щоб як читати, так і оновлювати дані.

```python
from multiprocessing import Process, RLock, current_process
from multiprocessing.sharedctypes import Value, Array
from ctypes import Structure, c_double
import logging

logger = logging.getLogger()
stream_handler = logging.StreamHandler()
logger.addHandler(stream_handler)
logger.setLevel(logging.DEBUG)


class Point(Structure):
    _fields_ = [('x', c_double), ('y', c_double)]


def modify(num: Value, string: Array, arr: Array):
    logger.debug(f'Started {current_process().name}')
    logger.debug(f"Change num: {num.value}")
    with num.get_lock():
        num.value **= 2
    logger.debug(f"to num: {num.value}")
    with string.get_lock():
        string.value = string.value.upper()
    with arr.get_lock():
        for a in arr:
            a.x **= 2
            a.y **= 2
    logger.debug(f'Done {current_process().name}')


if __name__ == '__main__':
    lock = RLock()
    number = Value(c_double, 1.5, lock=lock)
    string = Array('c', b'hello world', lock=lock)
    array = Array(Point, [(1, -6), (-5, 2), (2, 9)], lock=lock)

    p = Process(target=modify, args=(number, string, array))
    p2 = Process(target=modify, args=(number, string, array))
    p.start()
    p2.start()
    p.join()
    p2.join()
    print(number.value)
    print(string.value)
    print([(arr.x, arr.y) for arr in array])
```

Виведення буде:
```
Started Process-2
Change num: 1.5
to num: 2.25
Done Process-2
Started Process-1
Change num: 2.25
to num: 5.0625
Done Process-1
5.0625
b'HELLO WORLD'
[(1.0, 1296.0), (625.0, 16.0), (16.0, 6561.0)]
```

## Менеджер ресурсів

Вимогливіший до ресурсів, але й зручніший у використанні механізм обміну даними між процесами — це **Менеджер ресурсів**. 

Основна перевага — можливість працювати по всій мережі та реалізувати розподілені обчислення між кількома комп'ютерами в одній мережі, реалізація `Python-like` списків та словників.

Недоліки:

- Необхідність синхронізувати доступ до загальних ресурсів;
- Обмеження типів, що підтримуються;
- Складне API.

Розглянемо наступний приклад:

```python
from multiprocessing import Process, Manager, current_process
from random import randint
from time import sleep
import logging

logger = logging.getLogger()
stream_handler = logging.StreamHandler()
logger.addHandler(stream_handler)
logger.setLevel(logging.DEBUG)


def worker(delay, val: Manager):
    name = current_process().name
    logger.debug(f'Started: {name}')
    sleep(delay)
    val[name] = current_process().pid
    logger.debug(f'Done: {name}')


if __name__ == '__main__':
    with Manager() as manager:
        m = manager.dict()
        processes = []
        for i in range(5):
            pr = Process(target=worker, args=(randint(1, 3), m))
            pr.start()
            processes.append(pr)

        [pr.join() for pr in processes]
        print(m)
```
Виведення:
```
Started: Process-2
Started: Process-3
Started: Process-5
Started: Process-4
Started: Process-6
Done: Process-3
Done: Process-5
Done: Process-2
Done: Process-6
Done: Process-4
{'Process-3': 7444, 'Process-5': 15976, 'Process-2': 15564, 'Process-6': 18896, 'Process-4': 14244}
```

У цьому прикладі ми запустили п'ять процесів і додали до словника `m`, для кожного процесу його `pid` — ідентифікатор процесу. Все це було створено та управлялося менеджером `Manager`.

Але є важливе зауваження. 
- Проксі-об'єкти `Manager` **не можуть поширювати зміни**, внесені до об'єктів, що змінюються всередині контейнера. 
- Іншими словами, якщо у вас є об'єкт `manager.list()`, **будь-які зміни в самому керованому списку розповсюджуються на всі інші процеси**. 
- Але якщо у вас є **звичайний список Python всередині цього списку**, **будь-які зміни у внутрішньому списку не поширюються, тому що менеджер не має можливості виявити зміни.**

Щоб розповсюдити зміни, ви також повинні використовувати об'єкти `manager.list()` для вкладених списків. (необхідний Python 3.6 або вище) або вам потрібно безпосередньо змінити об'єкт `manager.list()` [(див. примітку)](https://docs.python.org/3.5/library/multiprocessing.html#multiprocessing.managers.SyncManager.list) .